
<center><h1>Speeding up the Backward Pass</h1> </center>


<center>
<p>Seq2Seq Testing - IWSLT2017</p>
</center>

[Trnasformer Implementation code from ](https://nlp.seas.harvard.edu/2018/04/03/attention.html):
   [Sasha Rush](http://rush-nlp.com/).


In [ ]:
!pip install GPUtil
!pip install -q torchdata==0.3.0 torchtext==0.12 spacy==3.2 altair GPUtil
!python3 -m spacy download de_core_news_sm
!python3 -m spacy download en_core_web_sm

In [ ]:
import os
from os.path import exists
import torch
import torch.nn as nn
from torch.nn.functional import log_softmax, pad
import math
import copy
import time
from torch.optim.lr_scheduler import LambdaLR
import pandas as pd
import altair as alt
from torchtext.data.functional import to_map_style_dataset
from torch.utils.data import DataLoader
from torchtext.vocab import build_vocab_from_iterator
import torchtext.datasets as datasets
import spacy
import GPUtil
import warnings
from torch.utils.data.distributed import DistributedSampler
import torch.distributed as dist
import torch.multiprocessing as mp
from torch.nn.parallel import DistributedDataParallel as DDP
import gc

# Set to False to skip notebook execution (e.g. for debugging)
warnings.filterwarnings("ignore")
RUN_EXAMPLES = True

In [ ]:
torch.cuda.empty_cache()

In [ ]:
# Some convenience helper functions used throughout the notebook


def is_interactive_notebook():
    return __name__ == "__main__"


def show_example(fn, args=[]):
    if __name__ == "__main__" and RUN_EXAMPLES:
        return fn(*args)


def execute_example(fn, args=[]):
    if __name__ == "__main__" and RUN_EXAMPLES:
        fn(*args)


class DummyOptimizer(torch.optim.Optimizer):
    def __init__(self):
        self.param_groups = [{"lr": 0}]
        None

    def step(self):
        None

    def zero_grad(self, set_to_none=False):
        None


class DummyScheduler:
    def step(self):
        None

In [ ]:
class EncoderDecoder(nn.Module):
    """
    A standard Encoder-Decoder architecture. Base for this and many
    other models.
    """

    def __init__(self, encoder, decoder, src_embed, tgt_embed, generator):
        super(EncoderDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed
        self.tgt_embed = tgt_embed
        self.generator = generator

    def forward(self, src, tgt, src_mask, tgt_mask):
        "Take in and process masked src and target sequences."
        return self.decode(self.encode(src, src_mask), src_mask, tgt, tgt_mask)

    def encode(self, src, src_mask):
        return self.encoder(self.src_embed(src), src_mask)

    def decode(self, memory, src_mask, tgt, tgt_mask):
        return self.decoder(self.tgt_embed(tgt), memory, src_mask, tgt_mask)

In [ ]:
class Generator(nn.Module):
    "Define standard linear + softmax generation step."

    def __init__(self, d_model, vocab):
        super(Generator, self).__init__()
        self.proj = nn.Linear(d_model, vocab)

    def forward(self, x):
        return log_softmax(self.proj(x), dim=-1)

## MNet

In [ ]:
import torch
import torch.nn as nn

import torch.nn.functional as F
import torch.optim as optim

import time
import math
m_shape = (512, 512) # EMB, EMB

class GradSaver_Compute_M(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x, sequentialOutput, saver):
        # print('in forward in grad saver')
        ctx.save_for_backward(x, saver, sequentialOutput)
        return sequentialOutput.clone().detach()
    
    

    @staticmethod
    def backward(ctx, gradients):
        global lst_sqs_sum
        
        x, saver, s = ctx.saved_tensors
        g = gradients
        saver.g = gradients.clone()

        l,b,d = x.shape

        # reshape
        x = x.reshape(l*b, d)
        s = s.reshape(l*b, d)
        g = g.reshape(l*b, d)

        K = math.ceil(l*b / d)
        pad_size = ( (K*d) - (b*l) ) # should be int
        pad_value = 0

        # now apply padding to x,s,g and reshape
        x_pad = torch.nn.functional.pad(x, (0,0,0,pad_size), 'constant', pad_value).reshape(K,d,d)
        s_pad = torch.nn.functional.pad(s, (0,0,0,pad_size), 'constant', pad_value).reshape(K,d,d)
        g_pad = torch.nn.functional.pad(g, (0,0,0,pad_size), 'constant', pad_value).reshape(K,d,d)

        delta = 1e-9
        perturbation = delta + torch.eye(d).to('cuda:1')
        saver.g = gradients.clone()

        gbar = []
        gbar = [torch.mm(g_pad[k,:,:], layerwisenorm_M(softmax_func_M(torch.linalg.solve(x_pad[k,:,:] + perturbation, s_pad[k,:,:])))) for k in range(K)]

        g_out = torch.cat(gbar)

        # unpad and reshape
        g_out = g_out[:g_out.shape[0]-pad_size,:].reshape(l,b,d)        
        return g_out, None, None

class GradSaver_As_Is(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x, sequentialOutput, saver):
        ctx.save_for_backward(x, saver, sequentialOutput)
        return sequentialOutput.clone().detach()

    @staticmethod
    def backward(ctx, gradients):
        global lst_sqs_sum
        
        x, saver, s = ctx.saved_tensors
        saver.g = gradients.clone()
        return gradients, None, None
    

class MNet_Transformer_As_Is(torch.nn.Module):
    def __init__(self, sequentialLayers, output_size):
        print('Creating MNet_Transformer')
        super(MNet_Transformer_As_Is, self).__init__()
        self.layers = []
        for l in sequentialLayers:
            self.layers.append(l)
        self.input_x = None
        self.layersOutput = []
        self.saver = torch.ones(output_size, dtype = self.layers[0].norm.a_2.data.dtype, requires_grad=True).to('cuda:1') # check dims of this (batch, output of M) 
        self.gradDiverge = GradSaver_As_Is.apply
        self.sequentialOutput = None


        # vars for custom_attn
        self.h = H
        self.d_k = MODEL_DIM // self.h
        self.linears = clones(nn.Linear(MODEL_DIM, MODEL_DIM), 4)
        self.dropout = nn.Dropout(p=0.1)
        self.self_attn_func = MultiHeadedAttention(H, MODEL_DIM)

        self.use_M = True

    def getLayersOutput(self, x, mask,):
        self.layersOutput = []

        if self.use_M:
            x_clone = x.clone().detach()
            x_clone.requires_grad = False
            self.input_x = x_clone
        else:
            self.input_x = x

        sublayer_ct = 0
        lo = self.input_x
        for l in self.layers:

            if 'sublayer' in str(type(l)).lower() and sublayer_ct == 0:
                lo = l(lo, lambda x: self.self_attn_func(x, x, x, mask))
                self.attn = l.attn_mat
                sublayer_ct += 1
            elif 'sublayer' in str(type(l)).lower() and sublayer_ct > 0:
                lo = l(lo, feed_forward_layer)
                sublayer_ct = 0
            elif 'feed' in str(type(l)).lower():
                feed_forward_layer = l # save for second sublayer
            else:
                print('ERROR: unexpected layer: ', l)

        return lo

    def forward(self, x, mask,):
        if self.use_M:
            x_clone = x.clone().detach()
            x_clone.requires_grad = False
            self.sequentialOutput = self.getLayersOutput(x_clone, mask,)
            if self.saver.shape != self.sequentialOutput.shape:
                self.saver = torch.ones(self.sequentialOutput.shape).to('cuda:1')

            t = self.gradDiverge(x, self.sequentialOutput.clone().detach(), self.saver)
        else:
            self.sequentialOutput = self.getLayersOutput(x, mask,)
            t = self.sequentialOutput
        return t

    def backwardHidden(self):
        self.sequentialOutput.backward(gradient = self.saver.g.clone().detach())
        
    def get_parameters(self):
        ps = []
        ps.append(self.layers[0].out_proj.weight) # self attn
        ps.append(self.layers[0].out_proj.bias)

        for l in self.layers:
            if hasattr(l, 'weight') and hasattr(l, 'bias'):
                ps.append(l.weight)
                ps.append(l.bias)
        return ps
    
class MNet_Transformer_Compute_M(torch.nn.Module):
    def __init__(self, sequentialLayers, output_size):
        print('Creating MNet_Transformer')
        super(MNet_Transformer_Compute_M, self).__init__()
        self.layers = []
        for l in sequentialLayers:
            self.layers.append(l)
        self.input_x = None
        self.layersOutput = []
        self.saver = torch.ones(output_size, dtype = self.layers[0].norm.a_2.data.dtype, requires_grad=True).to('cuda:1') # check dims of this (batch, output of M) 
        self.gradDiverge = GradSaver_Compute_M.apply
        self.sequentialOutput = None


        # vars for custom_attn
        self.h = H
        self.d_k = MODEL_DIM // self.h
        self.linears = clones(nn.Linear(MODEL_DIM, MODEL_DIM), 4)
        self.dropout = nn.Dropout(p=0.1)
        self.self_attn_func = MultiHeadedAttention(H, MODEL_DIM)
        self.use_M = True

    def getLayersOutput(self, x, mask,):
        self.layersOutput = []

        if self.use_M:
            x_clone = x.clone().detach()
            x_clone.requires_grad = False
            self.input_x = x_clone
        else:
            self.input_x = x

        sublayer_ct = 0
        lo = self.input_x
        for l in self.layers:

            if 'sublayer' in str(type(l)).lower() and sublayer_ct == 0:
                lo = l(lo, lambda x: self.self_attn_func(x, x, x, mask))
                self.attn = l.attn_mat
                sublayer_ct += 1
            elif 'sublayer' in str(type(l)).lower() and sublayer_ct > 0:
                lo = l(lo, feed_forward_layer)
                sublayer_ct = 0
            elif 'feed' in str(type(l)).lower():
                feed_forward_layer = l # save for second sublayer
            else:
                print('ERROR: unexpected layer: ', l)

        return lo

    def forward(self, x, mask,):
        if self.use_M:
            x_clone = x.clone().detach()
            x_clone.requires_grad = False
            self.sequentialOutput = self.getLayersOutput(x_clone, mask,)
            if self.saver.shape != self.sequentialOutput.shape:
                self.saver = torch.ones(self.sequentialOutput.shape).to('cuda:1')

            t = self.gradDiverge(x, self.sequentialOutput.clone().detach(), self.saver)
        else:
            self.sequentialOutput = self.getLayersOutput(x, mask,)
            t = self.sequentialOutput
        return t

    def backwardHidden(self):
        self.sequentialOutput.backward(gradient = self.saver.g.clone().detach())
        
    def get_parameters(self):
        ps = []
        ps.append(self.layers[0].out_proj.weight) # self attn
        ps.append(self.layers[0].out_proj.bias)

        for l in self.layers:
            if hasattr(l, 'weight') and hasattr(l, 'bias'):
                ps.append(l.weight)
                ps.append(l.bias)
        return ps

In [ ]:
def clones(module, N):
    "Produce N identical layers."
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

In [ ]:
class CustomSequential(nn.Module):
    def __init__(self, encoder_layers) -> None:
        super(CustomSequential, self).__init__()
        self.encoder_layers = encoder_layers
        modules = []
        for i in range(len(encoder_layers)):
            modules.append(encoder_layers[i].sublayer[0])
            modules.append(encoder_layers[i].feed_forward)
            modules.append(encoder_layers[i].sublayer[1])
        self.custom_sequential = nn.Sequential(*modules)

class Encoder_MNet_As_Is(nn.Module):
    "Core encoder is a stack of N layers"

    def __init__(self, layer, N, skipping_layer_idx, use_layers_M_inference=True):
        super(Encoder_MNet_As_Is, self).__init__()
        self.use_layers_M_inference = use_layers_M_inference
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)

        self.layer_idx_M1 = skipping_layer_idx
        self.mnet1 = MNet_Transformer_As_Is(CustomSequential(self.layers[skipping_layer_idx[0]:]).custom_sequential, (BATCH_SIZE, MODEL_DIM))

    def forward(self, x, mask):
        "Pass the input (and mask) through each layer in turn."
        for idx, layer in enumerate(self.layers):
            if idx == self.layer_idx_M1[0]:
                if self.use_layers_M_inference:
                    x = self.mnet1(x, mask)
                # else continue to next layer
            elif idx in self.layer_idx_M1:

                continue # already covered by the MNet
            else:
                x = layer(x, mask)
        return self.norm(x)
    

class Encoder_MNet_Compute_M(nn.Module):
    "Core encoder is a stack of N layers"

    def __init__(self, layer, N, skipping_layer_idx, use_layers_M_inference=True):
        super(Encoder_MNet_Compute_M, self).__init__()
        self.use_layers_M_inference = use_layers_M_inference
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)
        
        self.layer_idx_M1 = skipping_layer_idx
        self.mnet1 = MNet_Transformer_Compute_M(CustomSequential(self.layers[skipping_layer_idx[0]:]).custom_sequential, (BATCH_SIZE, MODEL_DIM))

    def forward(self, x, mask):
        "Pass the input (and mask) through each layer in turn."
        for idx, layer in enumerate(self.layers):
            if idx == self.layer_idx_M1[0]:
                if self.use_layers_M_inference:
                    x = self.mnet1(x, mask)
                # else continue to next layer
            elif idx in self.layer_idx_M1:

                continue # already covered by the MNet
            else:
                x = layer(x, mask)
        return self.norm(x)
    
class Encoder_Base(nn.Module):
    "Core encoder is a stack of N layers"

    def __init__(self, layer, N, use_layers_M_inference=True):
        super(Encoder_Base, self).__init__()
        self.use_layers_M_inference = use_layers_M_inference
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)

        self.layer_idx_M1 = [3,4,5]

    def forward(self, x, mask):
        "Pass the input (and mask) through each layer in turn."
        for idx, layer in enumerate(self.layers):
            if idx in self.layer_idx_M1 and not self.use_layers_M_inference:
                continue # skip these layers for logic segmentation experiment
            else:
                x = layer(x, mask)
        return self.norm(x)

In [ ]:
class LayerNorm(nn.Module):
    "Construct a layernorm module (See citation for details)."

    def __init__(self, features, eps=1e-6):
        super(LayerNorm, self).__init__()

        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        try:
            return self.a_2 * (x - mean) / (std + self.eps) + self.b_2
        except RuntimeError:
            return self.a_2.cuda('cuda:1') * (x - mean) / (std + self.eps) + self.b_2.cuda('cuda:1')
        
class LayerNorm_CUDA(nn.Module):
    "Construct a layernorm module (See citation for details)."

    def __init__(self, features, eps=1e-6):
        super(LayerNorm_CUDA, self).__init__()

        self.a_2 = nn.Parameter(torch.ones(features)).to('cuda:1')
        self.b_2 = nn.Parameter(torch.zeros(features)).to('cuda:1')
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)

        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2


In [ ]:
softmax_func_M = nn.Softmax(dim=-1)
layerwisenorm_M = LayerNorm_CUDA(m_shape)

In [ ]:
class SublayerConnection(nn.Module):
    """
    A residual connection followed by a layer norm.
    Note for code simplicity the norm is first as opposed to last.
    """

    def __init__(self, size, dropout):
        super(SublayerConnection, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)
        self.attn_mat = None

    def forward(self, x, sublayer):
        "Apply residual connection to any sublayer with the same size."

        imd = sublayer(self.norm(x))
        if type(imd) is tuple:
            self.attn_mat = imd[-1] # grab second output from MHAttn
        else:
            imd = [imd, -1]
        return x + self.dropout(imd[0]) # regular opeartion

In [ ]:
class EncoderLayer(nn.Module):
    "Encoder is made up of self-attn and feed forward (defined below)"

    def __init__(self, size, self_attn, feed_forward, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = self_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 2)
        self.size = size

    def forward(self, x, mask):
        "Follow Figure 1 (left) for connections."
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask))
        return self.sublayer[1](x, self.feed_forward)

In [ ]:
class Decoder(nn.Module):
    "Generic N layer decoder with masking."

    def __init__(self, layer, N):
        super(Decoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)

    def forward(self, x, memory, src_mask, tgt_mask):
        for layer in self.layers:
            x = layer(x, memory, src_mask, tgt_mask)
        return self.norm(x)

In [ ]:
class DecoderLayer(nn.Module):
    "Decoder is made of self-attn, src-attn, and feed forward (defined below)"

    def __init__(self, size, self_attn, src_attn, feed_forward, dropout):
        super(DecoderLayer, self).__init__()
        self.size = size
        self.self_attn = self_attn
        self.src_attn = src_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 3)

    def forward(self, x, memory, src_mask, tgt_mask):
        "Follow Figure 1 (right) for connections."
        m = memory
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, tgt_mask))
        x = self.sublayer[1](x, lambda x: self.src_attn(x, m, m, src_mask))
        return self.sublayer[2](x, self.feed_forward)

In [ ]:
def subsequent_mask(size):
    "Mask out subsequent positions."
    attn_shape = (1, size, size)
    subsequent_mask = torch.triu(torch.ones(attn_shape), diagonal=1).type(
        torch.uint8
    )
    return subsequent_mask == 0

In [ ]:
def attention(query, key, value, mask=None, dropout=None):
    "Compute 'Scaled Dot Product Attention'"
    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k)
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)
    p_attn = scores.softmax(dim=-1)
    if dropout is not None:
        p_attn = dropout(p_attn)
    return torch.matmul(p_attn, value), p_attn

In [ ]:
class MultiHeadedAttention(nn.Module):
    def __init__(self, h, d_model, dropout=0.1):
        "Take in model size and number of heads."
        super(MultiHeadedAttention, self).__init__()
        assert d_model % h == 0
        # We assume d_v always equals d_k
        self.d_k = d_model // h
        self.h = h
        self.linears = clones(nn.Linear(d_model, d_model), 4)
        self.attn = None # this is the output of softmax (before multiplied by V) ------------------ NOTE
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, query, key, value, mask=None):
        "Implements Figure 2"
        if mask is not None:
            # Same mask applied to all h heads.
            mask = mask.unsqueeze(1)
        nbatches = query.size(0)

        # 1) Do all the linear projections in batch from d_model => h x d_k
        query, key, value = [
            lin(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2)
            for lin, x in zip(self.linears, (query, key, value))
        ]

        # 2) Apply attention on all the projected vectors in batch.
        x, self.attn = attention(
            query, key, value, mask=mask, dropout=self.dropout
        )

        # 3) "Concat" using a view and apply a final linear.
        x = (
            x.transpose(1, 2)
            .contiguous()
            .view(nbatches, -1, self.h * self.d_k)
        )
        del query
        del key
        del value
        return self.linears[-1](x), self.attn

In [ ]:
class PositionwiseFeedForward(nn.Module):
    "Implements FFN equation."

    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        try:
            return self.w_2(self.dropout(self.w_1(x).relu()))
        except RuntimeError:
            return (self.w_2.cuda('cuda:1'))(self.dropout(self.w_1.cuda('cuda:1')(x).relu()))

In [ ]:
class Embeddings(nn.Module):
    def __init__(self, d_model, vocab):
        super(Embeddings, self).__init__()
        self.lut = nn.Embedding(vocab, d_model)
        self.d_model = d_model

    def forward(self, x):
        return self.lut(x) * math.sqrt(self.d_model)

In [ ]:
class PositionalEncoding(nn.Module):
    "Implement the PE function."

    def __init__(self, d_model, dropout, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(
            torch.arange(0, d_model, 2) * -(math.log(10000.0) / d_model)
        )
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer("pe", pe)

    def forward(self, x):
        x = x + self.pe[:, : x.size(1)].requires_grad_(False)
        return self.dropout(x)

In [ ]:
def make_model_As_Is(
    src_vocab, tgt_vocab, N=6, d_model=512, d_ff=2048, h=8, dropout=0.1, use_layers_M_inference = True, skipping_layer_idx = [3,4,5]
):
    "Helper: Construct a model from hyperparameters."
    c = copy.deepcopy
    attn = MultiHeadedAttention(h, d_model)
    ff = PositionwiseFeedForward(d_model, d_ff, dropout)
    position = PositionalEncoding(d_model, dropout)
    model = EncoderDecoder(
        Encoder_MNet_As_Is(EncoderLayer(d_model, c(attn), c(ff), dropout), N, skipping_layer_idx, use_layers_M_inference=use_layers_M_inference),
        Decoder(DecoderLayer(d_model, c(attn), c(attn), c(ff), dropout), N),
        nn.Sequential(Embeddings(d_model, src_vocab), c(position)),
        nn.Sequential(Embeddings(d_model, tgt_vocab), c(position)),
        Generator(d_model, tgt_vocab),
    )

    # This was important from their code.
    # Initialize parameters with Glorot / fan_avg.
    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)
    return model

def make_model_Compute_M(
    src_vocab, tgt_vocab, N=6, d_model=512, d_ff=2048, h=8, dropout=0.1, use_layers_M_inference = True, skipping_layer_idx = [3,4,5]
):
    "Helper: Construct a model from hyperparameters."
    c = copy.deepcopy
    attn = MultiHeadedAttention(h, d_model)
    ff = PositionwiseFeedForward(d_model, d_ff, dropout)
    position = PositionalEncoding(d_model, dropout)
    model = EncoderDecoder(
        Encoder_MNet_Compute_M(EncoderLayer(d_model, c(attn), c(ff), dropout), N, skipping_layer_idx, use_layers_M_inference=use_layers_M_inference),
        Decoder(DecoderLayer(d_model, c(attn), c(attn), c(ff), dropout), N),
        nn.Sequential(Embeddings(d_model, src_vocab), c(position)),
        nn.Sequential(Embeddings(d_model, tgt_vocab), c(position)),
        Generator(d_model, tgt_vocab),
    )

    # This was important from their code.
    # Initialize parameters with Glorot / fan_avg.
    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)
    return model

def make_model_Base(
    src_vocab, tgt_vocab, N=6, d_model=512, d_ff=2048, h=8, dropout=0.1, use_layers_M_inference = True,
):
    "Helper: Construct a model from hyperparameters."
    c = copy.deepcopy
    attn = MultiHeadedAttention(h, d_model)
    ff = PositionwiseFeedForward(d_model, d_ff, dropout)
    position = PositionalEncoding(d_model, dropout)
    model = EncoderDecoder(
        Encoder_Base(EncoderLayer(d_model, c(attn), c(ff), dropout), N, use_layers_M_inference=use_layers_M_inference),
        Decoder(DecoderLayer(d_model, c(attn), c(attn), c(ff), dropout), N),
        nn.Sequential(Embeddings(d_model, src_vocab), c(position)),
        nn.Sequential(Embeddings(d_model, tgt_vocab), c(position)),
        Generator(d_model, tgt_vocab),
    )

    # This was important from their code.
    # Initialize parameters with Glorot / fan_avg.
    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)
    return model

In [ ]:
class Batch:
    """Object for holding a batch of data with mask during training."""

    def __init__(self, src, tgt=None, pad=2):  # 2 = <blank>
        self.src = src
        self.src_mask = (src != pad).unsqueeze(-2)
        if tgt is not None:
            self.tgt = tgt[:, :-1]
            self.tgt_y = tgt[:, 1:]
            self.tgt_mask = self.make_std_mask(self.tgt, pad)
            self.ntokens = (self.tgt_y != pad).data.sum()

    @staticmethod
    def make_std_mask(tgt, pad):
        "Create a mask to hide padding and future words."
        tgt_mask = (tgt != pad).unsqueeze(-2)
        tgt_mask = tgt_mask & subsequent_mask(tgt.size(-1)).type_as(
            tgt_mask.data
        )
        return tgt_mask

In [ ]:
class TrainState:
    """Track number of steps, examples, and tokens processed"""

    step: int = 0  # Steps in the current epoch
    accum_step: int = 0  # Number of gradient accumulation steps
    samples: int = 0  # total # of examples used
    tokens: int = 0  # total # of tokens processed

In [ ]:
def run_epoch(
    data_iter,
    model,
    loss_compute,
    optimizer,
    scheduler,
    mode="train",
    accum_iter=1,
    train_state=TrainState(),
):
    """Train a single epoch"""
    start = time.time()
    total_tokens = 0
    total_loss = 0
    tokens = 0
    n_accum = 0
    for i, batch in enumerate(data_iter):
        out = model.forward(
            batch.src, batch.tgt, batch.src_mask, batch.tgt_mask
        )
        loss, loss_node = loss_compute(out, batch.tgt_y, batch.ntokens)
        
        # loss_node = loss_node / accum_iter
        if mode == "train" or mode == "train+log":
            loss_node.backward()
            train_state.step += 1
            train_state.samples += batch.src.shape[0]
            train_state.tokens += batch.ntokens
            if i % accum_iter == 0:
                optimizer.step()
                optimizer.zero_grad(set_to_none=True)
                n_accum += 1
                train_state.accum_step += 1
            scheduler.step()
            training_loss.append(loss)
        else:
            val_loss.append(loss)
        total_loss += loss
        total_tokens += batch.ntokens
        tokens += batch.ntokens
        if i % 40 == 1 and (mode == "train" or mode == "train+log"):
            lr = optimizer.param_groups[0]["lr"]
            elapsed = time.time() - start
            print(
                (
                    "Epoch Step: %6d | Accumulation Step: %3d | Loss: %6.2f "
                    + "| Tokens / Sec: %7.1f | Learning Rate: %6.1e"
                )
                % (i, n_accum, loss / batch.ntokens, tokens / elapsed, lr)
            )
            start = time.time()
            tokens = 0
        del loss
        del loss_node
        
    return total_loss / total_tokens, train_state

In [ ]:
def run_epoch_M(
    data_iter,
    model,
    loss_compute,
    optimizer,
    scheduler,
    mode="train",
    accum_iter=1,
    train_state=TrainState(),
    use_M=True
):
    """Train a single epoch"""
    start = time.time()
    total_tokens = 0
    total_loss = 0
    tokens = 0
    n_accum = 0
    for i, batch in enumerate(data_iter):
        out = model.forward(
            batch.src, batch.tgt, batch.src_mask, batch.tgt_mask
        )
        loss, loss_node = loss_compute(out, batch.tgt_y, batch.ntokens)
        training_loss.append(loss)
        if mode == "train" or mode == "train+log":
            loss_node.backward()


            # update M's ----------------------------------------------
            if use_M:
                model.encoder.mnet1.backwardHidden()


            train_state.step += 1
            train_state.samples += batch.src.shape[0]
            train_state.tokens += batch.ntokens
            if i % accum_iter == 0:
                optimizer.step()
                optimizer.zero_grad(set_to_none=True)
                n_accum += 1
                train_state.accum_step += 1

            scheduler.step()

        total_loss += loss
        total_tokens += batch.ntokens
        tokens += batch.ntokens
        if i % 40 == 1 and (mode == "train" or mode == "train+log"):
            lr = optimizer.param_groups[0]["lr"]
            elapsed = time.time() - start
            print(
                (
                    "Epoch Step: %6d | Accumulation Step: %3d | Loss: %6.2f "
                    + "| Tokens / Sec: %7.1f | Learning Rate: %6.1e"
                )
                % (i, n_accum, loss / batch.ntokens, tokens / elapsed, lr)
            )
            start = time.time()
            tokens = 0
        del loss
        del loss_node
        
    return total_loss / total_tokens, train_state

MNET ends

In [ ]:
def rate(step, model_size, factor, warmup):
    """
    we have to default the step to 1 for LambdaLR function
    to avoid zero raising to negative power.
    """
    if step == 0:
        step = 1
    return factor * (
        model_size ** (-0.5) * min(step ** (-0.5), step * warmup ** (-1.5))
    )

In [ ]:
class LabelSmoothing(nn.Module):
    "Implement label smoothing."

    def __init__(self, size, padding_idx, smoothing=0.0):
        super(LabelSmoothing, self).__init__()
        self.criterion = nn.KLDivLoss(reduction="sum")
        self.padding_idx = padding_idx
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.size = size
        self.true_dist = None

    def forward(self, x, target):
        assert x.size(1) == self.size
        true_dist = x.data.clone()
        true_dist.fill_(self.smoothing / (self.size - 2))
        true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
        true_dist[:, self.padding_idx] = 0
        mask = torch.nonzero(target.data == self.padding_idx)
        if mask.dim() > 0:
            true_dist.index_fill_(0, mask.squeeze(), 0.0)
        self.true_dist = true_dist
        return self.criterion(x, true_dist.clone().detach())

In [ ]:
class SimpleLossCompute:
    "A simple loss compute and train function."

    def __init__(self, generator, criterion):
        self.generator = generator
        self.criterion = criterion

    def __call__(self, x, y, norm):
        x = self.generator(x)
        sloss = (
            self.criterion(
                x.contiguous().view(-1, x.size(-1)), y.contiguous().view(-1)
            )
            / norm
        )
        return sloss.data * norm, sloss

In [ ]:
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    memory = model.encode(src, src_mask)
    ys = torch.zeros(1, 1).fill_(start_symbol).type_as(src.data)
    for i in range(max_len - 1):
        out = model.decode(
            memory, src_mask, ys, subsequent_mask(ys.size(1)).type_as(src.data)
        )
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.data[0]
        ys = torch.cat(
            [ys, torch.zeros(1, 1).type_as(src.data).fill_(next_word)], dim=1
        )
    return ys

In [ ]:
# Load spacy tokenizer models, download them if they haven't been
# downloaded already


def load_tokenizers():

    try:
        spacy_de = spacy.load("de_core_news_sm")
    except IOError:
        os.system("python -m spacy download de_core_news_sm")
        spacy_de = spacy.load("de_core_news_sm")

    try:
        spacy_en = spacy.load("en_core_web_sm")
    except IOError:
        os.system("python -m spacy download en_core_web_sm")
        spacy_en = spacy.load("en_core_web_sm")

    return spacy_de, spacy_en

In [ ]:
def tokenize(text, tokenizer):
    return [tok.text for tok in tokenizer.tokenizer(text)]


def yield_tokens(data_iter, tokenizer, index):
    for from_to_tuple in data_iter:
        yield tokenizer(from_to_tuple[index])

In [ ]:
def build_vocabulary(spacy_de, spacy_en):
    def tokenize_de(text):
        return tokenize(text, spacy_de)

    def tokenize_en(text):
        return tokenize(text, spacy_en)

    print("Building German Vocabulary ...")
    train, val, test = datasets.Multi30k(language_pair=("de", "en"))
    vocab_src = build_vocab_from_iterator(
        yield_tokens(train + val + test, tokenize_de, index=0),
        min_freq=2,
        specials=["<s>", "</s>", "<blank>", "<unk>"],
    )

    print("Building English Vocabulary ...")
    train, val, test = datasets.Multi30k(language_pair=("de", "en"))
    vocab_tgt = build_vocab_from_iterator(
        yield_tokens(train + val + test, tokenize_en, index=1),
        min_freq=2,
        specials=["<s>", "</s>", "<blank>", "<unk>"],
    )

    vocab_src.set_default_index(vocab_src["<unk>"])
    vocab_tgt.set_default_index(vocab_tgt["<unk>"])

    return vocab_src, vocab_tgt


def load_vocab(spacy_de, spacy_en):
    if not exists("vocab.pt") or True:
        vocab_src, vocab_tgt = build_vocabulary(spacy_de, spacy_en)
        torch.save((vocab_src, vocab_tgt), "vocab.pt")
    else:
        vocab_src, vocab_tgt = torch.load("vocab.pt")
    print("Finished.\nVocabulary sizes:")
    print(len(vocab_src))
    print(len(vocab_tgt))
    return vocab_src, vocab_tgt


if is_interactive_notebook():
    # global variables used later in the script
    spacy_de1, spacy_en1 = show_example(load_tokenizers)
    vocab_src1, vocab_tgt1 = show_example(load_vocab, args=[spacy_de1, spacy_en1])

In [ ]:
def collate_batch(
    batch,
    src_pipeline,
    tgt_pipeline,
    src_vocab,
    tgt_vocab,
    device,
    max_padding=128,
    pad_id=2,
    language_pairing = None # only need for iwslt
):
    bs_id = torch.tensor([0], device=device)  # <s> token id
    eos_id = torch.tensor([1], device=device)  # </s> token id
    src_list, tgt_list = [], []
    try:
        _src, _tgt = batch[0]
    except ValueError:
        batch = [(batch[i]['translation'][language_pairing[0]], batch[i]['translation'][language_pairing[1]]) for i in range(len(batch))]

    for (_src, _tgt) in batch:
        processed_src = torch.cat(
            [
                bs_id,
                torch.tensor(
                    src_vocab(src_pipeline(_src)),
                    dtype=torch.int64,
                    device=device,
                ),
                eos_id,
            ],
            0,
        )
        processed_tgt = torch.cat(
            [
                bs_id,
                torch.tensor(
                    tgt_vocab(tgt_pipeline(_tgt)),
                    dtype=torch.int64,
                    device=device,
                ),
                eos_id,
            ],
            0,
        )
        src_list.append(
            # warning - overwrites values for negative values of padding - len
            pad(
                processed_src,
                (
                    0,
                    max_padding - len(processed_src),
                ),
                value=pad_id,
            )
        )
        tgt_list.append(
            pad(
                processed_tgt,
                (0, max_padding - len(processed_tgt)),
                value=pad_id,
            )
        )

    src = torch.stack(src_list)
    tgt = torch.stack(tgt_list)
    return (src, tgt)

In [ ]:
from datasets import load_dataset # for huggingface datasets

def create_dataloaders(
    device,
    vocab_src,
    vocab_tgt,
    spacy_de,
    spacy_en,
    batch_size=12000,
    max_padding=128,
    is_distributed=False,
    language_pair = ('de', 'en'),
    dataset_func = datasets.IWSLT2017
):
    # def create_dataloaders(batch_size=12000):
    def tokenize_de(text):
        return tokenize(text, spacy_de)

    def tokenize_en(text):
        return tokenize(text, spacy_en)

    def collate_fn(batch):
        return collate_batch(
            batch,
            tokenize_de,
            tokenize_en,
            vocab_src,
            vocab_tgt,
            device,
            max_padding=max_padding,
            pad_id=vocab_src.get_stoi()["<blank>"],
            language_pairing=language_pair
        )
    dataset_func = datasets.IWSLT2017
    if 'iwslt' in str(dataset_func).lower():
        dataset = load_dataset("iwslt2017", f'iwslt2017-{language_pair[0]}-{language_pair[1]}')
        train_iter, valid_iter, test_iter = dataset['train'], dataset['validation'], dataset['test']

    else: # just multi30k 
        train_iter, valid_iter, test_iter = dataset_func(
            language_pair=language_pair
        )
    try:
        # raise Exception
        train_iter_map = to_map_style_dataset(
            train_iter
        )  # DistributedSampler needs a dataset len()
        train_sampler = (
            DistributedSampler(train_iter_map) if is_distributed else None
        )
        valid_iter_map = to_map_style_dataset(valid_iter)
        valid_sampler = (
            DistributedSampler(valid_iter_map) if is_distributed else None
        )
    except:
        train_iter_map = train_iter
        valid_iter_map = valid_iter

    train_dataloader = DataLoader(
        train_iter_map,
        batch_size=batch_size,
        shuffle=True,
        sampler=None,
        collate_fn=collate_fn,
    )
    valid_dataloader = DataLoader(
        valid_iter_map,
        batch_size=batch_size,
        shuffle=True,
        sampler=None,
        collate_fn=collate_fn,
    )
    return train_dataloader, valid_dataloader


def create_dataloader_test(
    device,
    vocab_src,
    vocab_tgt,
    spacy_de,
    spacy_en,
    batch_size=12000,
    max_padding=128,
    is_distributed=False,
    language_pair = ('de', 'en'),
    dataset_func = datasets.IWSLT2017
):
    # def create_dataloaders(batch_size=12000)        

    def tokenize_de(text):
        return tokenize(text, spacy_de)

    def tokenize_en(text):
        return tokenize(text, spacy_en)

    def collate_fn(batch):
        return collate_batch(
            batch,
            tokenize_de,
            tokenize_en,
            vocab_src,
            vocab_tgt,
            device,
            max_padding=max_padding,
            pad_id=vocab_src.get_stoi()["<blank>"],
            language_pairing=language_pair
        )
    dataset_func = datasets.IWSLT2017
    if 'iwslt' in str(dataset_func).lower():
        dataset = load_dataset("iwslt2017", f'iwslt2017-{language_pair[0]}-{language_pair[1]}')
        train_iter, valid_iter, test_iter = dataset['train'], dataset['validation'], dataset['test']
    else:
        train_iter, valid_iter, test_iter = dataset_func(
            language_pair=language_pair
        )

    try:
        test_iter_map = to_map_style_dataset(
            test_iter
        )  # DistributedSampler needs a dataset len()
        test_sampler = (
            DistributedSampler(test_iter_map) if is_distributed else None
        )
    except:
        test_iter_map = test_iter

    test_dataloader = DataLoader(
        test_iter_map,
        batch_size=batch_size,
        shuffle=True,
        sampler=None,
        collate_fn=collate_fn,
    )

    return test_dataloader

In [ ]:
training_loss = []
val_loss = []

def train_worker(
    gpu,
    ngpus_per_node,
    vocab_src,
    vocab_tgt,
    spacy_de,
    spacy_en,
    config,
    is_distributed=False,
    model_name = 'base', # base, as_is, compute_m
    dataset_func = datasets.Multi30k,
    language_pair=('de','en'),
    iter_num=-1,
    timestamp=time.time(),
    N = 6,
    skipping_layer_idx = [3,4,5]

):
    time_start = time.time()
    print(f"Train worker process using GPU: {gpu} for training", flush=True)
    torch.cuda.set_device(gpu)

    pad_idx = vocab_tgt["<blank>"]
    d_model = 512

    if model_name == 'base':
        model = make_model_Base(len(vocab_src), len(vocab_tgt), N=N)
    elif model_name == 'as_is':
        model = make_model_As_Is(len(vocab_src), len(vocab_tgt), N=N, skipping_layer_idx = skipping_layer_idx)
    elif model_name == 'compute_m':
        model = make_model_Compute_M(len(vocab_src), len(vocab_tgt), N=N, skipping_layer_idx = skipping_layer_idx)
    else:
        raise NotImplementedError('Model name not implemented: please use one of the following: [base, as_is, compute_m]')

    
    model.cuda(gpu)
    module = model
    is_main_process = True
    if is_distributed:
        dist.init_process_group(
            "nccl", init_method="env://", rank=gpu, world_size=ngpus_per_node
        )
        model = DDP(model, device_ids=[gpu])
        module = model.module
        is_main_process = gpu == 0

    criterion = LabelSmoothing(
        size=len(vocab_tgt), padding_idx=pad_idx, smoothing=0.1
    )
    criterion.cuda(gpu)

    train_dataloader, valid_dataloader = create_dataloaders(
        gpu,
        vocab_src,
        vocab_tgt,
        spacy_de,
        spacy_en,
        batch_size=config["batch_size"] // ngpus_per_node,
        max_padding=config["max_padding"],
        is_distributed=is_distributed,
        language_pair=language_pair,
        dataset_func=dataset_func
    )

    optimizer = torch.optim.Adam(
        model.parameters(), lr=config["base_lr"], betas=(0.9, 0.98), eps=1e-9
    )
    lr_scheduler = LambdaLR(
        optimizer=optimizer,
        lr_lambda=lambda step: rate(
            step, d_model, factor=1, warmup=config["warmup"]
        ),
    )
    train_state = TrainState()
    best_valid_loss = -999999
    for epoch in range(config["num_epochs"]):
        if is_distributed:
            train_dataloader.sampler.set_epoch(epoch)
            valid_dataloader.sampler.set_epoch(epoch)

        model.train()
        try:
            model.encoder.mnet1.use_M = True 
        except:
            pass
            
        print(f"[GPU{gpu}] Epoch {epoch} Training ====", flush=True)
        s = time.time()
        if model_name == 'as_is' or model_name == 'compute_m':
            _, train_state = run_epoch_M(
                (Batch(b[0], b[1], pad_idx) for b in train_dataloader),
                model,
                SimpleLossCompute(module.generator, criterion),
                optimizer,
                lr_scheduler,
                mode="train+log",
                accum_iter=config["accum_iter"],
                train_state=train_state,
                use_M = True,
            )
        else:
            _, train_state = run_epoch( # for base model
                (Batch(b[0], b[1], pad_idx) for b in train_dataloader),
                model,
                SimpleLossCompute(module.generator, criterion),
                optimizer,
                lr_scheduler,
                mode="train+log",
                accum_iter=config["accum_iter"],
                train_state=train_state,
            )
        print(f"Epoch training time: {time.time() - s}")

        GPUtil.showUtilization()
        if is_main_process:
            file_path = "%s%.2d.pt" % (config["file_prefix"], epoch)
            torch.save(module.state_dict(), file_path)
        torch.cuda.empty_cache()

        print(f"[GPU{gpu}] Epoch {epoch} Validation ====", flush=True)
        model.eval()
        sloss = run_epoch(
            (Batch(b[0], b[1], pad_idx) for b in valid_dataloader),
            model,
            SimpleLossCompute(module.generator, criterion),
            DummyOptimizer(),
            DummyScheduler(),
            mode="eval",
        )
        if best_valid_loss < 0:
            best_valid_loss = sloss[0]
        elif best_valid_loss > sloss[0]:
            best_valid_loss = sloss[0]
            print('best performance saved')
            file_path = f"%sfinal_best_val_loss_{model_name}_{N}_{timestamp}_trial_{iter_num}.pt" % config["file_prefix"]
            torch.save(module.state_dict(), file_path)
        print(sloss)
        torch.cuda.empty_cache()


    if is_main_process:
        otherinfo = {
            'experiment name' : 'E6D6 - last 3 layers of E are abstracted by M (if not base)',
            'total wallclock training time' : time.time()-time_start,
            'config_info' : config
        }
        file_path = f"%sfinal_{model_name}_{N}_{timestamp}_trial_{iter_num}.pt" % config["file_prefix"]
        torch.save(training_loss, f'saved_{model_name}_{N}_{timestamp}_trial_{iter_num}_train_losses.pt')
        torch.save(val_loss, f'saved_{model_name}_{N}_{timestamp}_trial_{iter_num}_val_losses.pt')
        torch.save(otherinfo, f'saved_{model_name}_{N}_{timestamp}_trial_{iter_num}_otherinfo.pt')
        torch.save(module.state_dict(), file_path)
        file_path1 = f"%sfinal.pt" % config["file_prefix"]
        torch.save(module.state_dict(), file_path1)

        del model
        torch.cuda.empty_cache()
        gc.collect()

        return file_path

In [ ]:
IS_CUDA = True
H = 8
BATCH_SIZE = 32
MODEL_DIM = 512
def load_trained_model(model_name, iter_num, timestamp, N = 6, skipping_layer_idx = [3,4,5], dataset_func = datasets.IWSLT2017):
    config = {
        "batch_size": 32,
        "distributed": False,
        "num_epochs": 14,
        "accum_iter": 10,
        "base_lr": 1.0,
        "max_padding": 72,
        "warmup": 3000,
        "file_prefix": "multi30k_model_",
    }
    model_path = "multi30k_model_final.pt"
    if not exists(model_path) or True: # default to always train
        model_fp = train_worker(
        1, 1, vocab_src, vocab_tgt, spacy_de, spacy_en, config, False, model_name=model_name, iter_num=iter_num, timestamp=timestamp,
        N = N, skipping_layer_idx = skipping_layer_idx 
    )
    print(f"Done training {model_name}, config_info={config}")
    return model_fp

s = time.time()


# Pipeline

In [ ]:
# run 5 training runs on 1 dataset (for now) and one language pairing (for now de->en), be sure to reference config in load_trained_model to adjust epochs


torch.cuda.empty_cache()
gc.collect()
GPUtil.showUtilization()

num_runs = 5
datasets_list = [datasets.IWSLT2017]
language_pairings = [('de', 'en')]
model_names = ['base','as_is', 'compute_m']

for j in range(len(datasets_list)):
    for lp in language_pairings:
        if lp == ('de', 'en'):
            global_lp_flag = lp
            spacy_de, spacy_en = spacy_de1, spacy_en1
            vocab_src, vocab_tgt = vocab_src1, vocab_tgt1
        else:
            global_lp_flag = lp
            spacy_de, spacy_en = spacy_de2, spacy_en2
            vocab_src, vocab_tgt = vocab_src2, vocab_tgt2
        for mn in model_names:
            print("Model Name", mn, 'trails starts ============================')
            timestamp = 5032023
            for i in range(num_runs):
                print('running', mn, 'N', 6)
                training_loss = []
                val_loss = []
                model_fp = load_trained_model(model_name=mn, iter_num=i, timestamp=timestamp, N = 6, skipping_layer_idx = [3,4,5]) # execute training and save model
                
                print(f'Trial: {i}. {mn} model trained successfully see file {model_fp} for model weights')
            print("Model Name", mn, 'trails ends ============================ \n\n')



# Testing

In [ ]:
from tqdm import tqdm
refs = []
preds = []
src = []


def check_outputs(
    valid_dataloader,
    model,
    vocab_src,
    vocab_tgt,
    n_examples=15,
    pad_idx=2,
    eos_string="</s>",
):
    results = [()] * n_examples

    for idx in tqdm(range(n_examples)):
        b = next(iter(valid_dataloader))
        rb = Batch(b[0], b[1], pad_idx)
        rb.src = rb.src.to('cuda:1')
        rb.src_mask = rb.src_mask.to('cuda:1')

        src_tokens = [
            vocab_src.get_itos()[x] for x in rb.src[0] if x != pad_idx
        ]
        
        # save the src for comet
        src.append(" ".join(src_tokens).replace("\n", ""))
        
        tgt_tokens = [
            vocab_tgt.get_itos()[x] for x in rb.tgt[0] if x != pad_idx
        ]


        refs.append(" ".join(tgt_tokens).replace("\n", ""))

        model_out = greedy_decode(model, rb.src, rb.src_mask, 72, 0)[0]
        model_txt = (
            " ".join(
                [vocab_tgt.get_itos()[x] for x in model_out if x != pad_idx]
            ).split(eos_string, 1)[0]
            + eos_string
        )
        preds.append(model_txt.replace("\n", ""))
        results[idx] = (rb, src_tokens, tgt_tokens, model_out, model_txt)
    return results

Make Predictions

In [ ]:
import glob
import os

do_test = True
path_root = './'
path_root_to_save = './'
N_sizes = [6]

if do_test:

    model_names = ['base','as_is', 'compute_m']
    language_pairings = [('de', 'en')]
    num_trials = 5


    for mn in model_names:
        for lp in language_pairings:

            # set global vocab vars
            if lp == ('de', 'en'):
                spacy_de, spacy_en = spacy_de1, spacy_en1
                vocab_src, vocab_tgt = vocab_src1, vocab_tgt1
            else:
                spacy_de, spacy_en = spacy_de2, spacy_en2
                vocab_src, vocab_tgt = vocab_src2, vocab_tgt2

            for i in range(num_trials):
                for N in N_sizes:
                    # find path
                    if mn != 'base' and N == 1:
                        continue
                    if N == 3:
                        skipping_layer_idx = [2]
                    else:
                        skipping_layer_idx = [3,4,5]
                        
                    paths = glob.glob(f'{path_root}/*')
                    p_ct = 0
                    for p in paths:
                        if f'{mn}_{N}' in p and f'best_val_loss' in p and f'trial_{i}.pt' in p:
                            path = p
                            p_ct += 1
                            print(path)
                    
                    assert p_ct == 1
                    
                    # reset lists
                    refs = []
                    preds = []
                    src = []

                    # init correct model
                    if mn == 'base':
                        model = make_model_Base(len(vocab_src), len(vocab_tgt), N=N)
                    elif mn == 'as_is':
                        model = make_model_As_Is(len(vocab_src), len(vocab_tgt), N=N, skipping_layer_idx = skipping_layer_idx)
                    elif mn == 'compute_m':
                        model = make_model_Compute_M(len(vocab_src), len(vocab_tgt), N=N, skipping_layer_idx = skipping_layer_idx)

                    # load model weights
                    print("Loading model weights from ", path)
                    model.load_state_dict(torch.load(path))
                    model = model.to('cuda:1')
                    model.eval()

                    test_dataloader = create_dataloader_test(
                            torch.device("cuda:1"),
                            vocab_src,
                            vocab_tgt,
                            spacy_de,
                            spacy_en,
                            batch_size=1,
                            is_distributed=False,
                        )
                    print(len(test_dataloader))
                    
                    print("Checking Model Outputs:")
                    example_data_M = check_outputs(
                        test_dataloader, model, vocab_src, vocab_tgt, n_examples= len(test_dataloader) # limit this n_examples for faster computation
                    )


                    # create dir if not exists
                    if not os.path.exists(path_root_to_save):
                        os.makedirs(path_root_to_save)
                    lp_str = f'{lp[0]}->{lp[1]}'
                    # save the preds, refs, src for this specific model 
                    fo = open(f'{path_root_to_save}/model={mn}_N={N}_lang_pair={lp_str}_trial={i}_preds.txt', 'w')
                    for sentence in preds:
                        if len(sentence) == 1:
                            sentence = sentence[0]
                        sentence = (sentence.replace('<s>', '').replace('</s>', '')).strip()
                        fo.write("".join(sentence) + '\n')
                        # print("".join(sentence))
                    fo.close()

                    fo = open(f'{path_root_to_save}/model={mn}_N={N}_lang_pair={lp_str}_trial={i}_refs.txt', 'w')
                    for sentence in refs:
                        if len(sentence) == 1:
                            sentence = sentence[0]
                        sentence = (sentence.replace('<s>', '').replace('</s>', '')).strip()
                        fo.write("".join(sentence) + '\n')
                        # print("".join(sentence))
                    fo.close()

                    fo = open(f'{path_root_to_save}/model={mn}_N={N}_lang_pair={lp_str}_trial={i}_src.txt', 'w')
                    for sentence in src:
                        if len(sentence) == 1:
                            sentence = sentence[0]
                        sentence = (sentence.replace('<s>', '').replace('</s>', '')).strip()
                        fo.write("".join(sentence) + '\n')
                        # print("".join(sentence))
                    fo.close()

                    print('\nDone writing preds, refs, and src files for this model')

Compute Prediction Scores

In [ ]:
try:
    import torchmetrics
except:
    !pip install torchmetrics

try:
    import evaluate
except:
    !pip install evaluate

from torchmetrics import BLEUScore, SacreBLEUScore
import evaluate
import numpy as np
import pandas as pd

In [ ]:
def readfile(fn, data):
    with open(fn) as file:
        while line := file.readline():
            data.append(line.rstrip())
            # break
    return data

model_names = ['base', 'as_is', 'compute_m']
language_pairings = [('de', 'en')]
num_trials = 5
N_lists = ['6']
path_root = './'
paths = glob.glob(f'{path_root}/*')

df_dict = {}
for mn in model_names:
    for lp in language_pairings:
        for N in N_lists:
            if mn != 'base' and N =='1':
                continue
            lp_str = f'{lp[0]}->{lp[1]}'
            df_dict[f'{mn}_{lp_str}_{N}'] = {
                'BLEU' : (0,0), # format key as tuple (mean, std)
                'SacreBLEU' : (0,0),
                'METEOR' : (0,0),
                'COMET' : (0,0)
            }
            metrics_dict = {
                'bleu' : [],
                'sacrebleu' : [],
                'meteor' : [],
                'comet' : []
            }
            for i in range(num_trials):
                
                p_ct = 0
                for p in paths:
                    if mn in p and f'trial={i}' in p and f'N={N}' in p:
                        if 'preds' in p:
                            preds_path = p
                        elif 'refs' in p:
                            refs_path = p
                        elif 'src' in p:
                            src_path = p
                        p_ct += 1
                        print(p)
                assert p_ct == 3

                # read data into lists
                preds = []
                refs = []
                src = []
                readfile(preds_path, preds)
                readfile(refs_path, refs)
                readfile(src_path, src)

                # bleu
                if len(refs[0]) > 1:
                    print('Adding surrounding list for each target')
                    refs_bleu = [[x] for x in refs]
                else:
                    refs_bleu = refs

                bleu = BLEUScore()
                torch_bleu = bleu(preds, refs_bleu)
                metrics_dict['bleu'].append(torch_bleu)

                # sacrebleu
                if len(refs[0]) > 1:
                    print('Adding surrounding list for each target')
                    refs_sacrebleu = [[x] for x in refs]
                else:
                    refs_sacrebleu = refs

                sacre_bleu = SacreBLEUScore(tokenize='none')
                torch_sacrebleu = sacre_bleu(preds, refs_sacrebleu)
                metrics_dict['sacrebleu'].append(torch_sacrebleu)

                # meteor
                meteor = evaluate.load('meteor')
                results = meteor.compute(predictions=preds, references=refs)
                huggingface_meteor = (round(results['meteor'], 2))
                metrics_dict['meteor'].append(huggingface_meteor)

            # compute mean and std over trials and put in dict
            mean_bleu = np.mean(np.array(metrics_dict['bleu']))
            std_bleu = np.std(np.array(metrics_dict['bleu']))

            mean_sacrebleu = np.mean(np.array(metrics_dict['sacrebleu']))
            std_sacrebleu = np.std(np.array(metrics_dict['sacrebleu']))

            mean_meteor = np.mean(np.array(metrics_dict['meteor']))
            std_meteor = np.std(np.array(metrics_dict['meteor']))

            # mean_comet = np.mean(np.array(metrics_dict['comet']))
            # std_comet = np.std(np.array(metrics_dict['comet']))


            df_dict[f'{mn}_{lp_str}_{N}'] = {
                'BLEU' : (mean_bleu,std_bleu), # format key as tuple (mean, std)
                'SacreBLEU' : (mean_sacrebleu,std_sacrebleu),
                'METEOR' : (mean_meteor,std_meteor),
                'COMET' : (0,0) # disabled for now
            }

results_df = pd.DataFrame(df_dict)
results_df
